In [1]:
import pandas as pd
import numpy as np

import os
wd = os.getcwd()
wd

'C:\\Users\\Nils\\Documents\\KIT-TVWL\\Master\\SS 18\\KD Seminar\\Git\\mlp'

In [2]:
df = pd.read_csv("../final_data/complete_data.csv")
df.Timestamp = pd.to_datetime(df.Timestamp)
df = df.drop(["Unnamed: 0"], axis=1)
df.head()

,Timestamp,ID,articleCount,avgSentiment,stdSentiment,25quantileSentiment,50quantileSentiment,75quantileSentiment,maxSentiment,minSentiment,Previous_Day_Return,Next_Day_Return
0,2018-05-02,BASF,156,0.190012,0.236579,0.119930,0.236800,0.323370,0.65250,-0.63234,0.011060,-0.017219
1,2018-05-02,Tesla,510,0.068932,0.173565,-0.070315,0.072627,0.202486,0.64572,-0.50300,0.004093,-0.057051
2,2018-05-02,Google,1901,0.106414,0.192993,-0.005088,0.069690,0.248100,0.98770,-0.55366,0.025714,-0.017464
3,2018-05-02,Allianz,121,-0.052265,0.264638,-0.230230,-0.094850,0.091920,0.58215,-0.66297,0.013549,-0.009181
4,2018-05-02,Airbus,250,0.055327,0.230707,-0.014180,0.030780,0.195380,0.60256,-0.57493,0.007473,0.000612


In [3]:
#df = df.drop_duplicates(subset=["Timestamp", "ID"])

In [4]:
df.shape
nr_days = int(df.shape[0] / 10)
nr_feats = df.shape[1] -2 #(-2 wegen Timestamp & ID)
print("Days {}".format(nr_days))
print("Features {}".format(nr_feats))

Days 11.0
Features 10


In [30]:
df["Timestamp"].value_counts()

2018-05-04    10
2018-05-16    10
2018-05-09    10
2018-05-02    10
2018-05-14    10
2018-05-17    10
2018-05-10    10
2018-05-03    10
2018-05-15    10
2018-05-08    10
2018-05-11    10
Name: Timestamp, dtype: int64

In [31]:
df.head()

,Timestamp,ID,articleCount,avgSentiment,stdSentiment,25quantileSentiment,50quantileSentiment,75quantileSentiment,maxSentiment,minSentiment,Previous_Day_Return,Next_Day_Return
0,2018-05-02,BASF,156,0.190012,0.236579,0.119930,0.236800,0.323370,0.65250,-0.63234,0.011060,-0.017219
1,2018-05-02,Tesla,510,0.068932,0.173565,-0.070315,0.072627,0.202486,0.64572,-0.50300,0.004093,-0.057051
2,2018-05-02,Google,1901,0.106414,0.192993,-0.005088,0.069690,0.248100,0.98770,-0.55366,0.025714,-0.017464
3,2018-05-02,Allianz,121,-0.052265,0.264638,-0.230230,-0.094850,0.091920,0.58215,-0.66297,0.013549,-0.009181
4,2018-05-02,Airbus,250,0.055327,0.230707,-0.014180,0.030780,0.195380,0.60256,-0.57493,0.007473,0.000612


In [32]:
X = []#np.empty((int(nr_days) ,int(nr_feats * 10)))

In [33]:
for d in df["Timestamp"].value_counts().index:
    X.append(df[df["Timestamp"] == d].drop(["Timestamp", "ID"], axis=1).values.reshape(1,nr_feats*10))
    

In [34]:
X = np.array(X).reshape(int(nr_days), nr_feats*10)
X.shape

(11, 100)

In [42]:
df_labels = df[["Timestamp","Next_Day_Return"]].copy()
df_labels["Next_Day_Return"].values.reshape(int(nr_days), 10)

array([[-0.01721889, -0.05705099, -0.01746398, -0.00918132,  0.00061175,
         0.00181067,  0.00350719, -0.0202173 ,  0.        , -0.01004057],
       [ 0.03848335,  0.00773992, -0.0080557 , -0.0065317 ,  0.03332835,
         0.01892551,  0.01262541, -0.02097312,  0.00640083,  0.00847805],
       [ 0.0041417 , -0.01048699, -0.00348867, -0.00353219,  0.00479514,
         0.00792103, -0.00264577,  0.00279818, -0.002593  ,  0.01339733],
       [-0.0328532 , -0.01084348, -0.00882565,  0.02501474,  0.00290647,
         0.00411967,  0.00021751, -0.01747101,  0.00701645,  0.01603135],
       [ 0.01420268,  0.00434028,  0.01365875,  0.00533603, -0.00598168,
        -0.03328691,  0.0099655 ,  0.01445442,  0.01429254,  0.00468751],
       [ 0.00380743, -0.0019508 , -0.005275  , -0.00910477, -0.00260702,
        -0.0013062 , -0.00583092, -0.01306777, -0.00585368, -0.00765923],
       [ 0.00449527,  0.00448108, -0.00126706, -0.00575948, -0.02313133,
        -0.03065852, -0.02483841, -0.00233583

In [154]:
df_labels_single = df_labels.groupby("Timestamp").agg("last")

In [155]:
df_labels_single.head()

,005930.KS,VIV,ALV.DE,TSS,BAYN.DE,TSLA,AIR.DE,AAPL,BAS.DE,BMW.DE
Timestamp,,,,,,,,,,
2018-05-02,0.000000,-0.010665,0.013549,-0.013838,0.003419,0.004093,0.007473,0.043227,0.011060,0.006682
2018-05-03,0.000000,-0.020217,-0.009181,0.008611,0.003507,-0.057051,0.000612,0.001811,-0.017219,-0.010041
2018-05-04,-0.020973,-0.008056,0.008478,0.014659,0.018926,0.033328,0.006401,0.038483,0.012625,-0.006532
2018-05-08,0.013397,-0.010487,-0.003489,0.010593,-0.003532,-0.002646,-0.002593,0.004795,0.002798,0.004142
2018-05-09,-0.032853,-0.017471,-0.008826,0.005824,0.004120,0.016031,-0.010843,0.007016,0.002906,0.000218


In [156]:
Y = df_labels_single.values
Y.shape

(9, 10)

In [157]:
from keras.models import Sequential
from keras.layers import Dense, Activation

from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [158]:
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.15, random_state=42)

In [160]:
dims = X.shape[1]
print(dims, 'dims')
print("Building model...")

nb_classes = Y.shape[1]
print(nb_classes, 'classes')

model = Sequential()
model.add(Dense(nb_classes, input_shape=(dims,), activation='sigmoid'))
model.add(Dense(nb_classes**2))
model.add(Dense(nb_classes**2))
model.add(Dense(nb_classes))
model.add(Activation('softplus'))
model.add(Activation('softmax'))
model.compile(optimizer='sgd', loss='categorical_crossentropy')
model.fit(X, Y, validation_data = (X_val, Y_val), epochs=10, 
          batch_size=128, verbose=True)

190 dims
Building model...
10 classes
Train on 9 samples, validate on 2 samples
Epoch 1/10
9/9 [==============================] - 1s 58ms/step - loss: -0.0452 - val_loss: -0.2786
Epoch 2/10
9/9 [==============================] - 0s 556us/step - loss: -0.0452 - val_loss: -0.2787
Epoch 3/10
9/9 [==============================] - 0s 445us/step - loss: -0.0453 - val_loss: -0.2787
Epoch 4/10
9/9 [==============================] - 0s 444us/step - loss: -0.0453 - val_loss: -0.2788
Epoch 5/10
9/9 [==============================] - 0s 1ms/step - loss: -0.0453 - val_loss: -0.2789
Epoch 6/10
9/9 [==============================] - 0s 1ms/step - loss: -0.0454 - val_loss: -0.2789
Epoch 7/10
9/9 [==============================] - 0s 334us/step - loss: -0.0454 - val_loss: -0.2790
Epoch 8/10
9/9 [==============================] - 0s 1ms/step - loss: -0.0455 - val_loss: -0.2791
Epoch 9/10
9/9 [==============================] - 0s 553us/step - loss: -0.0455 - val_loss: -0.2791
Epoch 10/10
9/9 [==========

In [135]:
X[0]

array([  5.51000000e+02,   8.21914827e-02,   2.06793140e-01,
        -1.01450000e-02,   4.14600000e-02,   2.13785000e-01,
         7.24430000e-01,  -6.32340000e-01,   0.00000000e+00,
        -2.09731234e-02,  -8.05570083e-03,   8.47804836e-03,
         1.46592056e-02,   1.89255091e-02,   3.33283517e-02,
         6.40083466e-03,   3.84833470e-02,   1.26254066e-02,
        -6.53170186e-03,   3.51000000e+02,  -6.38005924e-05,
         2.54183595e-01,  -1.81920000e-01,  -3.94000000e-02,
         2.19880000e-01,   6.17980000e-01,  -6.99633333e-01,
         0.00000000e+00,  -2.09731234e-02,  -8.05570083e-03,
         8.47804836e-03,   1.46592056e-02,   1.89255091e-02,
         3.33283517e-02,   6.40083466e-03,   3.84833470e-02,
         1.26254066e-02,  -6.53170186e-03,   2.04900000e+03,
         7.03780595e-02,   2.13552659e-01,  -2.62300000e-02,
         3.96700000e-02,   2.10740000e-01,   7.28870000e-01,
        -9.91700000e-01,   0.00000000e+00,  -2.09731234e-02,
        -8.05570083e-03,

In [161]:
E = model.predict(X[0].reshape(1,190))
E

array([[ 0.15545654,  0.07820908,  0.15075111,  0.07553826,  0.06204455,
         0.06763228,  0.08510923,  0.11990739,  0.1234183 ,  0.08193333]], dtype=float32)

In [162]:
Y[0]

array([ 0.        , -0.01066487,  0.01354925, -0.01383775,  0.00341881,
        0.00409271,  0.00747303,  0.04322714,  0.01106002,  0.0066825 ])

In [139]:
EY[0]

array([[ 0.04923758,  0.05990248,  0.12029236,  0.14767935,  0.1304228 ,
         0.04514487,  0.12636857,  0.00601043,  0.12278158,  0.1271591 ]])